<a href="https://colab.research.google.com/github/T0kiniaina/State_Of_Art/blob/main/State_of_Art_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scholarly

In [ ]:
import time
import random
from scholarly import scholarly
import pandas as pd

# Define your search query
query = "machine learning"

# Search Google Scholar
search_query = scholarly.search_pubs(query)

# Initialize an empty list to store the results
results = []

# Iterate through the search results
for i in range(3):  # Get the top 10 results
    try:
        paper = next(search_query)

        # Extraer los campos relevantes del paper
        title = paper.get('bib', {}).get('title', 'Unknown')
        author = paper.get('bib', {}).get('author', 'Unknown')
        year = paper.get('bib', {}).get('pub_year', 'Unknown')
        citations = paper.get('num_citations', 'Unknown')
        doi = paper.get('pub_url', 'Unknown')  # DOI usualmente está en 'pub_url'
        institution = paper.get('source', 'Unknown')  # Fuente o institución de publicación

        # Append the result
        results.append({
            "title": title,
            "author": author,
            "year": year,
            "citations": citations,
            "doi": doi,
            "institution": institution
        })

        # Añadir un retardo aleatorio entre 5 y 15 segundos
        sleep_time = random.uniform(5, 15)
        print(f"Sleeping for {sleep_time:.2f} seconds")
        time.sleep(sleep_time)

    except StopIteration:
        break

# Create a pandas DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame
df


In [ ]:
import time
import random
from scholarly import scholarly
import pandas as pd

# Define your search query
query = "machine learning"

# Search Google Scholar
search_query = scholarly.search_pubs(query)

# Initialize an empty list to store the results
results = []

# Iterate through the search results
for i in range(3):  # Get the top 10 results
    try:
        paper = next(search_query)

        # Extraer los campos relevantes del paper
        title = paper.get('bib', {}).get('title', 'Unknown')
        authors = paper.get('bib', {}).get('author', 'Unknown')
        year = paper.get('bib', {}).get('pub_year', 'Unknown')
        citations = paper.get('num_citations', 'Unknown')
        doi = paper.get('pub_url', 'Unknown')  # DOI o URL al paper
        journal = paper.get('bib', {}).get('journal', 'Unknown')  # Revista o conferencia
        abstract = paper.get('bib', {}).get('abstract', 'Unknown')  # Resumen del paper
        keywords = paper.get('bib', {}).get('keywords', 'Unknown')  # Palabras clave (si están disponibles)
        institution = paper.get('source', 'Unknown')  # Fuente o institución de publicación
        pdf_url = paper.get('eprint_url', doi)  # Enlace al texto completo si está disponible

        # Append the result
        results.append({
            "title": title,
            "authors": authors,
            "year": year,
            "citations": citations,
            "doi": doi,
            # "journal/conference": journal,
            "abstract": abstract,
            # "keywords": keywords,
            # "institution": institution,
            # "pdf_url": pdf_url
        })

        # Añadir un retardo aleatorio entre 5 y 15 segundos
        sleep_time = random.uniform(5, 15)
        print(f"Sleeping for {sleep_time:.2f} seconds")
        time.sleep(sleep_time)

    except StopIteration:
        break

# Create a pandas DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame
df


In [ ]:
!pip install scholarly

In [ ]:
import time
import random
from scholarly import scholarly
import pandas as pd
from tenacity import retry, stop_after_attempt, wait_random  # For retry mechanism
import itertools  # To cycle through proxies

# Retry configuration: retry up to 3 times with random wait between 5 and 10 seconds
@retry(stop=stop_after_attempt(3), wait=wait_random(min=5, max=10))
def fetch_paper(paper, proxy):
    # Extract relevant fields from the paper
    title = paper.get('bib', {}).get('title', 'Unknown')
    authors = paper.get('bib', {}).get('author', 'Unknown')
    year = paper.get('bib', {}).get('pub_year', 'Unknown')
    citations = paper.get('num_citations', 'Unknown')
    doi = paper.get('pub_url', 'Unknown')  # DOI or URL for the paper
    institution = paper.get('source', 'Unknown')  # Source or institution
    abstract = paper.get('bib', {}).get('abstract', 'Unknown')
    journal = paper.get('bib', {}).get('venue', 'Unknown')
    volume = paper.get('bib', {}).get('volume', 'Unknown')
    issue = paper.get('bib', {}).get('issue', 'Unknown')
    pages = paper.get('bib', {}).get('pages', 'Unknown')

    return {
        "Title": title,
        "Author(s)": authors,
        "Year": year,
        "Citations": citations,
        "DOI": doi,
        "Institution": institution,
        "Abstract": abstract,
        "Journal/Conference Name": journal,
        "Volume": volume,
        "Issue": issue,
        "Pages": pages,
        "Proxy Used": proxy  # Track the proxy used
    }

# Define a list of proxies
proxies = [
    {"http": "http://123.45.67.89:8080", "https": "http://123.45.67.89:8080"},  # Replace with real IP
    {"http": "http://98.76.54.32:3128", "https": "http://98.76.54.32:3128"},    # Replace with real IP
    {"http": "http://11.22.33.44:80", "https": "http://11.22.33.44:80"},        # Replace with real IP
]

# Cycle through proxies
proxy_cycle = itertools.cycle(proxies)

# Define the list of search queries (keywords)
keywords = ["Image processing", "Thermal Image", "GDP"]

# Initialize an empty list to store the results
results = []

# Loop through each keyword
for keyword in keywords:
    print(f"Searching for keyword: {keyword}")
    search_query = scholarly.search_pubs(keyword)  # Update the search query for each keyword

    # Fetch up to 10 results for each keyword with proxy rotation
    for i in range(3):
        try:
            paper = next(search_query)
            current_proxy = next(proxy_cycle)  # Get the next proxy from the cycle
            result = fetch_paper(paper, current_proxy)  # Fetch and retry if needed, with the current proxy
            result["Keyword"] = keyword  # Add the keyword to the result
            results.append(result)

            # Random sleep to avoid rate limiting (can be reduced if needed)
            sleep_time = random.uniform(3, 5)
            print(f"Sleeping for {sleep_time:.2f} seconds, using proxy: {current_proxy}")
            time.sleep(sleep_time)

        except StopIteration:
            break

# Create a pandas DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame or save it to a CSV file
# df.to_csv('google_scholar_results_with_proxies_keywords.csv', index=False)
# print(df)
df

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Save the DataFrame to your Google Drive folder
folder_path = '/content/drive/My Drive/Data_analytics/Projects/20241008_state_of_art/'  # Replace 'your_folder_name' with the actual folder name in your Drive
file_name = 'google_scholar.csv'
file_path = folder_path + file_name

df.to_csv(file_path, index=False, encoding='utf-8-sig')
print(f"File saved to: {file_path}")

In [ ]:
!apt-get install git

In [ ]:
!git config --global user.email "frodrigueza.career@example.com"
!git config --global user.name "Mike-R0d"

In [ ]:
!git clone https://github.com/Mike-R0d/State_of_Art.git

In [ ]:
%cd State_of_Art


In [ ]:
# Save the dataframe in the GitHub folder
file_path = '/content/State_of_Art/google_scholar_results.csv'
df.to_csv(file_path, index=False)
